# Citibike Ridership Data Import

This notebook performs initial data processing on Citibike ridership data to prepare it for time series analysis. There are two main steps:
- Read in monthly ride level data, process it (aggregate it to daily data by station ID, drop columns), and concatenate
- Append neighborhood and borough level information to each station

In [1]:
# Imports

import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from geopy.geocoders import Nominatim
from shapely.geometry import Point, Polygon, shape
import geopandas as gpd
import json

In [2]:
# Import util functions to pre-process the data

from util.preprocess_util import pre_process, pre_process_2021, convert_station, get_neighborhood

## Functions

In [3]:
def neighborhood_json(point):
    '''
    Function accepts a Point object from the shapely library.
    It parses through the JSON of nyc neighborhood geo data, checking if any of them contain the point.
    If there is a match, the neighborhood name is returned.
    
    '''
    for feature in nycmap['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return feature['properties']['ntaname']
            continue

In [4]:
def borough_json(point):
    '''
    This is a repeat of the function above, except to return borough instead of neighborhood.
    
    Function accepts a Point object from the shapely library.
    It parses through the JSON of nyc neighborhood geo data, checking if any of them contain the point.
    If there is a match, the borough name is returned.
    
    '''
    for feature in nycmap['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return feature['properties']['boro_name']
            continue

## Read and process initial ridership level data

- The files are read in partitioned based on common file and column name structure
    - June 2013 standalone
    - July 2013 through August 2014
    - Sept 2014 through Sept 2016
    - Oct 2016 through March 2017 (pass in different column names)
    - April 2017 through Jan 2021
    - Feb 2021 through Oct 2021

In [9]:
# Define time threshold to ensure there are no rides under 60 seconds

time_threshold = timedelta(seconds=60)

In [10]:
# Define standard column names to be used throughout

standard_cols = ['tripduration', 'starttime', 'stoptime', 'start station id', 'start station name', 
                 'start station latitude', 'start station longitude', 'end station id', 'end station name', 
                 'end station latitude', 'end station longitude', 'bikeid', 'usertype', 'birth year', 'gender']

In [11]:
# Read in June 2013

df_201306 = pd.read_csv('./ridership_raw/201306-citibike-tripdata.csv')

In [12]:
# Preprocess June 2013 dataset

df_201306 = pre_process(df_201306)

df_201306.to_csv('./processed/201306.csv')

In [7]:
# Read in July 2013 through August 2014

# df_list1 = []

month = 7
year = 2013

for i in range(14):
    
    if month > 12:
        month = 1
        year = 2014
    
    if month >= 10:
        df = pd.read_csv(f"./ridership_raw/{year}-{month} - Citi Bike trip data.csv")
        df = pre_process(df)
        df.to_csv(f"./processed/{year}{month}.csv")
    else:
        df = pd.read_csv(f"./ridership_raw/{year}-0{month} - Citi Bike trip data.csv")
        df = pre_process(df)
        df.to_csv(f"./processed/{year}0{month}.csv")
    
    month += 1

In [8]:
# Read in Sept 2014 through Sept 2016

# df_list2 = []

month = 9
year = 2014

for i in range(25):
    
    if month > 12:
        month = 1
        year += 1
    
    if month >= 10:
        df = pd.read_csv(f"./ridership_raw/{year}{month}-citibike-tripdata.csv")
        df = pre_process(df)
        df.to_csv(f"./processed/{year}{month}.csv")
    else:
        df = pd.read_csv(f"./ridership_raw/{year}0{month}-citibike-tripdata.csv")
        df = pre_process(df)
        df.to_csv(f"./processed/{year}0{month}.csv")
    month += 1

In [9]:
# Read in one of the files to make sure it's exported correctly

test = pd.read_csv('./processed/201409.csv')

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9748 entries, 0 to 9747
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   starttime                9748 non-null   object 
 1   start station id         9748 non-null   int64  
 2   start station latitude   9748 non-null   float64
 3   start station longitude  9748 non-null   float64
 4   ride_count               9748 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 380.9+ KB


In [10]:
# Read in Oct 2016 through March 2017 with different column names

# df_list3 = []

month = 10
year = 2016

for i in range(6):
    
    if month > 12:
        month = 1
        year += 1
    
    if month >= 10:
        df = pd.read_csv(f"./ridership_raw/{year}{month}-citibike-tripdata.csv", names=standard_cols, skiprows=1)
        df = pre_process(df)
        df.to_csv(f"./processed/{year}{month}.csv")

    else:
        df = pd.read_csv(f"./ridership_raw/{year}0{month}-citibike-tripdata.csv", names=standard_cols, skiprows=1)
        df = pre_process(df)
        df.to_csv(f"./processed/{year}0{month}.csv")
    
    month += 1

In [11]:
# Read in April 2017 through Jan 2021

# df_list4 = []

month = 4
year = 2017

for i in range(46):
    
    if month > 12:
        month = 1
        year += 1
    
    if month >= 10:
        df = pd.read_csv(f"./ridership_raw/{year}{month}-citibike-tripdata.csv")
        df = pre_process(df)
        df.to_csv(f"./processed/{year}{month}.csv")
    else:
        df = pd.read_csv(f"./ridership_raw/{year}0{month}-citibike-tripdata.csv")
        df = pre_process(df)
        df.to_csv(f"./processed/{year}0{month}.csv")
    
    month += 1

In [56]:
# Read in remainder through Oct 2021

# df_list5 = []

month = 2
year = 2021

for i in range(9):
    
    if month > 12:
        month = 1
        year += 1
    
    if month >= 10:
        df = pd.read_csv(f"./ridership_raw/{year}{month}-citibike-tripdata.csv", low_memory=False)
        df = pre_process_2021(df)
        df.to_csv(f"./processed/{year}{month}.csv")
    else:
        df = pd.read_csv(f"./ridership_raw/{year}0{month}-citibike-tripdata.csv", low_memory=False)
        df = pre_process_2021(df)
        df.to_csv(f"./processed/{year}0{month}.csv")
    
    month += 1

## Consolidate ridership files

Read in the clean files, append them together, and rename the columns

In [5]:
# Read in each CSV and append to the dataframe list

df_list = []
month = 6
year = 2013

for i in range(101):
    
    if month > 12:
        month = 1
        year += 1
    
    if month >= 10:
        df = pd.read_csv(f"./processed/{year}{month}.csv", index_col=0)
        df_list.append(df)

    else:
        df = pd.read_csv(f"./processed/{year}0{month}.csv", index_col=0)
        df_list.append(df)
    
    month += 1

In [6]:
# Concat list of dataframes together

df = pd.concat(df_list)

In [7]:
# Make sure index is a datetime variable

df.index = pd.to_datetime(df.index)
df['year'] = df.index.map(lambda x: x.year)

In [8]:
# Rename columns

df = df.rename(columns={'start station id': 'station_id', 'start station latitude': 'lat', 
                                    'start station longitude': 'long'})

In [9]:
# Sense check output

df.head()

,station_id,lat,long,ride_count,year
starttime,,,,,
2013-06-01,72,40.767272,-73.993929,40,2013
2013-06-01,79,40.719116,-74.006667,61,2013
2013-06-01,82,40.711174,-74.000165,6,2013
2013-06-01,83,40.683826,-73.976323,32,2013
2013-06-01,116,40.741776,-74.001497,53,2013


In [10]:
# Sense check output

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2144526 entries, 2013-06-01 to 2021-10-31
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   station_id  object 
 1   lat         float64
 2   long        float64
 3   ride_count  int64  
 4   year        int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 98.2+ MB


## Append neighborhood and borough information

There are two approaches to adding neighborhood information:
- geo_json file from NYC Open Data. Contains polygon objects containing coordinate information for each borough / neighborhood. Neighborhoods are matched based on Citibike station coordinate data. This is the approach that yields the best results (almost all stations match)
- geopy reverse encoder: this library takes coordinates and returns information about the neighborhood. It works reasonably well, but produces an unacceptable number of nulls (~20% of stations). Code for this approach shown below, but ultimately note used

### geo_json approach

In [11]:
# Find unique set of stations

df_unique = df.drop_duplicates(subset='station_id')

df_stations = df_unique[['station_id', 'lat', 'long']].reset_index().drop('starttime', axis=1)

df_stations.info()

# There may be some "duplicate stations" given that the IDs may not be consistently named

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3849 entries, 0 to 3848
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   station_id  3849 non-null   object 
 1   lat         3849 non-null   float64
 2   long        3849 non-null   float64
dtypes: float64(2), object(1)
memory usage: 90.3+ KB


In [12]:
# Read in JSON map of NYC neighborhoods

nycmap = json.load(open('./nyc_geo_data/2010 Neighborhood Tabulation Areas (NTAs).geojson'))

In [13]:
# Explore structure of JSON
# Each 'feature' has the coordinate map, borough name, and nta_name, which is what we're interested in

nycmap['features'][0]

{'type': 'Feature',
 'properties': {'ntacode': 'QN51',
  'shape_area': '52488277.4492',
  'county_fips': '081',
  'ntaname': 'Murray Hill',
  'shape_leng': '33266.9048559',
  'boro_name': 'Queens',
  'boro_code': '4'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-73.80379022888246, 40.77561011179248],
     [-73.80098974064948, 40.77538911645844],
     [-73.79865192006208, 40.77520055898499],
     [-73.79853009427278, 40.77519072888339],
     [-73.7982329551194, 40.77484830546582],
     [-73.7978540045323, 40.77441566944928],
     [-73.79772190077279, 40.774272373809396],
     [-73.79757522894504, 40.77413684204422],
     [-73.79741530994487, 40.77401038228574],
     [-73.79741522772021, 40.774010319151415],
     [-73.79741514502729, 40.774010265062664],
     [-73.79730005603771, 40.77393228605373],
     [-73.79724339821908, 40.77389389663614],
     [-73.79672685097921, 40.77360922870773],
     [-73.79652490521445, 40.773503278471615],
     [-73.79571707389985, 40.7730696

In [14]:
# Create a 'point' variable column out of the longitute and latitutde

df_stations['point_coordinates'] = df_stations.apply(lambda row: Point(row['long'], row['lat']), axis=1)

df_stations.head()

/Users/jeffreymarvel/opt/anaconda3/envs/citibike/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,station_id,lat,long,point_coordinates
0,72,40.767272,-73.993929,POINT (-73.99392888 40.76727216)
1,79,40.719116,-74.006667,POINT (-74.00666661 40.71911552)
2,82,40.711174,-74.000165,POINT (-74.00016545 40.71117416)
3,83,40.683826,-73.976323,POINT (-73.97632328 40.68382604)
4,116,40.741776,-74.001497,POINT (-74.00149746 40.74177603)


In [15]:
# Apply neighborhood function to the coordinates

df_stations['neighborhood'] = df_stations['point_coordinates'].apply(lambda x: neighborhood_json(x))

In [17]:
# Apply borough function to the coordinates

df_stations['borough'] = df_stations['point_coordinates'].apply(lambda x: borough_json(x))

In [18]:
# Sense check the Borough output

df_stations['borough'].value_counts()

Manhattan    1624
Brooklyn     1091
Bronx         650
Queens        474
Name: borough, dtype: int64

In [19]:
# Sense check the neighborhood output

df_stations['neighborhood'].value_counts()

Astoria                                       134
Bushwick South                                130
Hudson Yards-Chelsea-Flatiron-Union Square    114
Midtown-Midtown South                         107
Mott Haven-Port Morris                         94
                                             ... 
park-cemetery-etc-Queens                        9
Bay Ridge                                       8
Kensington-Ocean Parkway                        7
Flatbush                                        2
Borough Park                                    2
Name: neighborhood, Length: 82, dtype: int64

In [20]:
# Map these stations back to the main dataframe based on station_id

df = df.merge(df_stations, left_on='station_id', right_on='station_id', how='left').set_index(df.index)

df.drop(labels=['lat_y', 'long_y', 'point_coordinates'], axis=1, inplace=True)

df.rename(columns={'lat_x': 'lat', 'long_x': 'long'}, inplace=True)

In [21]:
# Sense check output

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2144526 entries, 2013-06-01 to 2021-10-31
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   station_id    object 
 1   lat           float64
 2   long          float64
 3   ride_count    int64  
 4   year          int64  
 5   neighborhood  object 
 6   borough       object 
dtypes: float64(2), int64(2), object(3)
memory usage: 130.9+ MB


In [25]:
# Read to CSV

df.to_csv('./clean_data/clean_ridershare_data.csv')

# Next Notebook: Citibike EDA

https://github.com/marvelje/citibike_ridership_project/blob/main/citibike_eda.ipynb

### Geopy approach

- Note: commented out as it's not in use, but want to display an alternate approach

In [ ]:
# Simplify down to only lat and long

# df_lat_long = df_stations[['lat', 'long']]

# Convert to geopandas object

# gdf_citibike = gpd.GeoDataFrame(df_lat_long, geometry=gpd.points_from_xy(df_lat_long.long, df_lat_long.lat))

In [ ]:
# Test on sample lat /long

# geolocator = Nominatim(user_agent='citi_bike_share_analysis')

# sample_lat_long = df_stations.loc[0,['lat', 'long']]
# lat = sample_lat_long[0]
# long = sample_lat_long[1]
# combined = str(lat) + ', ' + str(long)
# location = geolocator.reverse(combined)
# location.raw['address']['neighbourhood']

In [ ]:
# Apply geolocator to every station in the dataset

# df_stations['neighborhood'] = df_stations.apply(lambda row: get_neighborhood(row['lat'], row['long']), axis=1)

In [ ]:
# Display null neighborhoods from this approach

# null_neighborhoods = df_stations[df_stations['neighborhood'].isna()]

# null_neighborhoods